In [5]:
from models import upvote_predictor
import importlib
importlib.reload(upvote_predictor)
from util import artifacts, cache
importlib.reload(artifacts)
import prediction
importlib.reload(prediction)

MINI MODE: True


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


<module 'prediction' from '/Users/cameron/Documents/mlx/hackernews/prediction.py'>

In [6]:
items_table = "hacker_news.items"

hn_test_posts = cache.query("hn_posts_for_test", f"""SELECT
    title,
    karma,
    score
    FROM {items_table}
    INNER JOIN hacker_news.users u ON {items_table}.by = u.id
    WHERE type = 'story' AND title IS NOT null
    OFFSET 1000000
    LIMIT 10
""")

hn_test_posts.dropna(inplace=True)

def predict_row(row):
    return prediction.predict(row['title'], row['karma'])

hn_test_posts['predicted_score'] = hn_test_posts.apply(predict_row, axis=1)
hn_test_posts.head(10)

,title,karma,score,predicted_score
0,"Oracle buys KSplice, shoots users in the foot",3342,2,1.670995
1,Change The Virtual Machine Date in Virtual Box...,1,1,1.098999
2,‘Kaantham’ - A film with a message,1,1,0.746261
3,Top 10: Swarmbots,1282,2,1.262890
4,For developers: How to build a business model ...,13,8,1.036162
5,Show HN: GeekRations,1641,4,1.807838
6,"What's going to be the next ""really big"" thing?",71,1,1.055585
7,All your Favorite Cities in 8-Bit,5305,1,1.771759
8,500 reasons to vote against Mitt Romney and fo...,12,7,0.971087
9,'A Universe From Nothing' by Lawrence Krauss [...,7909,1,1.279634
